<a href="https://colab.research.google.com/github/imPhacko/PublicProjects/blob/main/Projektas_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = "/content/drive/MyDrive/DATA AI/TRUMPxBIDEN.txt"
#raw_text = open(filename, "r", encoding="utf-8").read()
raw_text = open(filename, "r", encoding='cp1252').read()
raw_text = raw_text.lower()
print(raw_text[:200])

moderator: (00:18)
good evening, everyone. good evening. thank you so much for being here. it is such an honor for me to moderate this debate tonight, the final debate. i want to welcome the first fam


In [ ]:
# decode py2 compat.
#raw_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(raw_text)} characters')

Length of text: 229998 characters


In [ ]:
print(raw_text[:5000])

moderator: (00:18)
good evening, everyone. good evening. thank you so much for being here. it is such an honor for me to moderate this debate tonight, the final debate. i want to welcome the first family and the first lady. we’re so glad and thankful that you are feeling better. i want to welcome the biden family, dr. jill biden. thank you all for being here tonight. we are so excited. we’re looking forward to a really robust discussion. and the only thing i would reiterate are the cpd guidelines that when the candidates are talking, please hold any applause or any other reactions. except of course, when they walk out, make sure you cheer and loud and applause so that everyone can hear you. thank you for having me. this is really the honor of a lifetime. i am going to sit down and just get organized and get settled and the show will start very soon. thank you for being here. (silence). good evening from belmont university in nashville, tennessee. i’m moderator of nbc news. and i welcom

In [ ]:
vocab = sorted(set(raw_text))
print(f'{len(vocab)} unique characters')

57 unique characters


In [ ]:
##pd.Series(len(x) for x in ' '.join(df.vocab).split()).value_counts().sort_index().plot(kind='bar', figsize=(12, 3))

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[27, 28, 29, 30, 31, 32, 33], [50, 51, 52]]>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(raw_text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(229998,), dtype=int64, numpy=array([39, 41, 30, ..., 47, 10,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

m
o
d
e
r
a
t
o
r
:


In [ ]:
seq_length = 100
examples_per_epoch = len(raw_text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'm' b'o' b'd' b'e' b'r' b'a' b't' b'o' b'r' b':' b' ' b'(' b'0' b'0'
 b':' b'1' b'8' b')' b'\n' b'g' b'o' b'o' b'd' b' ' b'e' b'v' b'e' b'n'
 b'i' b'n' b'g' b',' b' ' b'e' b'v' b'e' b'r' b'y' b'o' b'n' b'e' b'.'
 b' ' b'g' b'o' b'o' b'd' b' ' b'e' b'v' b'e' b'n' b'i' b'n' b'g' b'.'
 b' ' b't' b'h' b'a' b'n' b'k' b' ' b'y' b'o' b'u' b' ' b's' b'o' b' '
 b'm' b'u' b'c' b'h' b' ' b'f' b'o' b'r' b' ' b'b' b'e' b'i' b'n' b'g'
 b' ' b'h' b'e' b'r' b'e' b'.' b' ' b'i' b't' b' ' b'i' b's' b' ' b's'
 b'u' b'c' b'h'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'moderator: (00:18)\ngood evening, everyone. good evening. thank you so much for being here. it is such'
b' an honor for me to moderate this debate tonight, the final debate. i want to welcome the first famil'
b'y and the first lady. we\xe2\x80\x99re so glad and thankful that you are feeling better. i want to welcome the b'
b'iden family, dr. jill biden. thank you all for being here tonight. we are so excited. we\xe2\x80\x99re looking f'
b'orward to a really robust discussion. and the only thing i would reiterate are the cpd guidelines tha'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Donald Trump"))

(['D', 'o', 'n', 'a', 'l', 'd', ' ', 'T', 'r', 'u', 'm'],
 ['o', 'n', 'a', 'l', 'd', ' ', 'T', 'r', 'u', 'm', 'p'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'moderator: (00:18)\ngood evening, everyone. good evening. thank you so much for being here. it is suc'
Target: b'oderator: (00:18)\ngood evening, everyone. good evening. thank you so much for being here. it is such'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of vocab
vocab_size = len(vocab)

# embedding dimension
embedding_dim = 256

# RNN unit amount
rnn_units = 1024

In [ ]:
class Modelis(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = Modelis(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 58) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "modelis"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  14848     
                                                                 
 gru_3 (GRU)                 multiple                  3938304   
                                                                 
 dense_3 (Dense)             multiple                  59450     
                                                                 
Total params: 4,012,602
Trainable params: 4,012,602
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([34, 20, 33, 29,  5, 20, 14,  0, 52, 36, 50,  8,  5, 21, 18, 26,  2,
       13, 20, 20, 35, 32,  3, 42, 38, 28, 19, 34, 38, 12, 39, 52, 14,  3,
       57,  8, 47,  4, 45, 41, 31, 37, 44, 26, 31, 48, 28, 46, 40, 44, 44,
       49, 12, 41, 49, 21, 32,  2, 46, 40,  5, 19, 48, 50, 11, 14,  1, 19,
       41,  7, 22, 13, 21, 31, 12,  0, 56,  5, 45, 22, 48, 12,  6,  8, 14,
       32, 18, 27, 49, 13, 21, 13, 33, 39, 10,  7,  0,  7, 32, 29])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'. and with regard to more divided the nation, it can\xe2\x80\x99t stay divided. we can\xe2\x80\x99t be this way. and speak'

Next Char Predictions:
 b'h8gc&82[UNK]zjx,&96] 188if$plb7hl0mz2$\xe2\x80\xa6,u%soekr]evbtnrrw0ow9f tn&7vx/2\n7o):19e0[UNK]\xe2\x80\x9d&s:v0(,2f6aw191gm.)[UNK])fc'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 58)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.0597, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

57.95692

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Training

In [ ]:
EPOCHS = 40

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
35/35 [==============================] - 5s 134ms/step - loss: 0.4387
Epoch 2/40
35/35 [==============================] - 5s 132ms/step - loss: 0.4031
Epoch 3/40
35/35 [==============================] - 5s 135ms/step - loss: 0.3716
Epoch 4/40
35/35 [==============================] - 5s 131ms/step - loss: 0.3377
Epoch 5/40
35/35 [==============================] - 5s 133ms/step - loss: 0.3073
Epoch 6/40
35/35 [==============================] - 5s 133ms/step - loss: 0.2811
Epoch 7/40
35/35 [==============================] - 5s 133ms/step - loss: 0.2576
Epoch 8/40
35/35 [==============================] - 5s 133ms/step - loss: 0.2392
Epoch 9/40
35/35 [==============================] - 5s 132ms/step - loss: 0.2211
Epoch 10/40
35/35 [==============================] - 5s 133ms/step - loss: 0.2055
Epoch 11/40
35/35 [==============================] - 5s 133ms/step - loss: 0.1933
Epoch 12/40
35/35 [==============================] - 5s 134ms/step - loss: 0.1807
Epoch 13/40
35/35 [======

## Generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['moderator: '])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

moderator: (50:48)
all right. this is a guy who in fact need the he pood because you don’t have to come back every year.” “we have to come back.”

donald trump: (18:25)
he’s going to pask the coronavirus. president trump, the first question is for you. you’ll each have one minute, starting with you, mr. trump.

donald trump: (42:08)
well, chank within we knew. we got the masks. we made the vent lank in the stame, but you do that.

joe biden: (14:20)
because they were kere, and he plans hose back to the questions obamacare, both community.

donald trump: (24:46)
he tried it in his state.

joe biden: (14:59)
number two.

donald trump: (56:03)
now, having a get fire drag actuary of went and their first under obama asked for because i said, “i think you need more.” and just god demenations and presidential now, by the way, was given it. i do. i do it to them all. and by the way, my son…

moderator: (46:19)
wait a minute. y. gever people are dying. and he sumpond i’m going to eliminated, it